In [55]:
from Bio import SeqIO
import itertools
import numpy as np
import cooler
import hicstraw
from dataclasses import dataclass
from pathlib import Path
from cooltools.api.eigdecomp import cis_eig
from typing import List
from scipy.sparse import coo_matrix
import polars as pl

In [111]:
@dataclass
class Region:
    chrom1: str
    start1: int
    end1: int

    def __str__(self):
        return f"{self.chrom}:{self.start}-{self.end}" if self.start and self.end else chrom

def hicstrawContactRecordsToDict(records_list: List[hicstraw.contactRecord]):
    
    return df
    

def matrix_hic(filename: str, resolution: int, region: CisRegion, data_type: str, balance: str, sparse: bool):
    if balance is None:
        balance = "NONE"
    h = hicstraw.HiCFile(filename)
    mzd = h.getMatrixZoomData(region.chrom, region.chrom, data_type, balance, "BP", resolution)
    if sparse:
        # list of hicstraw.contactRecord objects which have binX, binY, counts params
        records = mzd.getRecords(region.start, region.end, region.start, region.end)
        d = {"start":[], "end":[], "count":[]}
        for record in records:
            d["start"].append(record.binX)
            d["end"].append(record.binY)
            d["count"].append(record.counts)
        return d
    else:
        # numpy.ndarray
        return mzd.getRecordsAsMatrix(region.start, region.end, region.start, region.end)

def matrix_mcool(filename, resolution, region, balance, sparse):
    c = cooler.Cooler(f"{filename}::/resolutions/{resolution}")

    # sparse
        # as_pixels = False: scipy.sparse.__coo.coo_matrix
        # as_pixels = True, join = False: pandas dataframe with columns bin1_id, bin2_id, count
        # as_pixels = True, join = True: pandas dataframe with columns chrom1, start1, end1, chrom2, start2, end2, count
            # balanced = True: one additional column "balanced"
    # not sparse
        # numpy.ndarray
    return c.matrix(sparse=sparse, balance=balance, as_pixels=True, join=True).fetch(str(region))

def matrix(filename: str, resolution: int, region: CisRegion, balance: str, sparse: bool):
    suffix = Path(filename).suffix

    if suffix == ".mcool":
        return matrix_mcool(filename, resolution, region, balance, sparse)
    elif suffix == ".hic":
        return matrix_hic(filename, resolution, region, "observed", balance, sparse)
    raise Exception(f"Suffix {suffix} not supported by dense_matrix loading function")


r = 100000
s = 1000000
e = 2000000
#c = matrix("KO_1_dsmin.hic", r, CisRegion("1", s, e), None, True)

mzd = hicstraw.HiCFile("KO_1_dsmin.hic").getMatrixZoomData("1", "2", "observed", "NONE", "BP", 1000000)
records = mzd.getRecords(0, int(1.1E6), 0, int(1.1E6))
c = records[0]
print(len(records), c.binX, c.binY, c.counts)
# At 1Mb resolution, there are two contacts at (1E6, 1E6)
# At 100kb resolution on 1:0-1.1E6, 2:0-1.1E6, the single contact is at (1E6, .3E6)
# At 10kb resolution, it is at (1.06E6, .3E6)
# At 1kb resolution, it is at (1.069E6, .309E6)
# Therefore it seems the binX is rounded down and binY are being rounded down to the resolution and represent start positions

2 1000000 0 2.0


In [102]:


def corr_pos(a, b):
    def ok(v):
        return not np.isnan(v) and v is not None

    keep_a = []
    keep_b = []
    for ai, bi in zip(a, b):
        if ok(ai) and ok(bi):
            keep_a.append(ai)
            keep_b.append(bi)
    
    return np.corrcoef(keep_a, keep_b)[0][1] > 0





def cis_eigs_gc_corrected(filename, res, chrom, gc_dist, eigs):
    mx = dense_matrix(filename, res, chrom)
    eigval, eigvec = cis_eig(mx, n_eigs = max(eigs)+1)
    for eig in eigs:
        if not corr_pos(eigvec[eig], gc_dist):
            eigvec[eig] *= -1
    return {eig:eigvec[eig] for eig in eigs}

@dataclass
class GCDist:
    gc: list[float]
    start: list[int]
    end: list[int]
    

def gc_dist(seq_it):
    g_dist = []
    start = 0
    starts = []
    ends = []
    while batch := tuple(itertools.islice(seq_it, None, 100000)):
        frac_g = (batch.count("G") + batch.count("G"))/len(batch)
        g_dist.append(frac_g)
        end = start + len(batch)
        starts.append(start)
        ends.append(end)
        start = ends[-1] + 1
    return GCDist(g_dist, starts, ends)

def write_compartment_scores(matrix_file, reference_fasta, resolutions, eigs_files):
    for record in SeqIO.parse(reference_fasta, "fasta"):
        gc = gc_dist(iter(record.seq))

        for res in resolutions:
            compartments = cis_eigs_gc_corrected(matrix_file, res, record.id, gc.gc, eigs.keys())
            print(compartments)
        break
write_compartment_scores("Mock.mcool", "hg38.fna", [100000])
            
            
        



# cs = compartment_score("Mock.mcool", 100000, "chr1", g_dist)
# import pyBigWig
# with pyBigWig.open("compartment.bw", "w") as bw:
#     header = [("chr1", len(cs))]
#     bw.addHeader(header)
#     contig = ["chr1"]*len(cs)
#     bw.addEntries(contig, starts = starts, ends = ends, values = cs)

{0: array([        nan,         nan,         nan, ..., -0.2110005 ,
        0.51287652,         nan]), 1: array([        nan,         nan,         nan, ..., -0.79052555,
       -1.21164271,         nan]), 2: array([        nan,         nan,         nan, ..., -0.4984122 ,
       -0.52792261,         nan])}


In [97]:
!pip install pybigwig


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.3/188.3 kB 5.2 MB/s eta 0:00:000:00:01
